In [77]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

from datetime import datetime, date, timedelta
from time import sleep
import numpy as np
import pandas as pd
import re
from re import search

In [60]:
# Scraper Tool
def scrape():
    temp = []
    time = datetime.now().strftime("%m/%d/%Y %H:%M:%S")
    # element to get all of the content
    contents =  driver.find_elements_by_class_name("tip-item")
    # element to get the star ratings
    items = driver.find_elements_by_xpath("/html/body/div[5]/div[2]/div[4]/div[*]/div[2]/ul") 

    rating = []
    for item in items:
        sub_items = item.find_elements(By.TAG_NAME, 'svg')
        rating_temp = []
        for sub_item in sub_items:
            rating_temp.append(sub_item.get_attribute("class"))
        rating.append(rating_temp.count("full-star"))

    for i, content in enumerate(contents[8:]):
            temp.append(content.text + f"\n{time}"+ f"\n{rating[i]}")

    return temp

# BJ's Data Collection

In [119]:
# PATH = "C:\Program Files (x86)\chromedriver.exe"
# driver = webdriver.Chrome(PATH)
# driver.get("https://www.groupon.com/deals/n-bjs-wholesale-club-memberships") #BJ's
# driver.maximize_window()

# sleep(4)

# all_reviews = driver.find_element_by_xpath("/html/body/section[2]/div/section/div/div[2]/div[2]/section/div/div/article/div[3]/div[2]/div[2]/a")

# all_reviews.click()

In [121]:
# temp = []
# for i in range(538): #538 Pages
    
#     for j in scrape():
#         temp.append(j)
#     next_button = driver.find_element_by_xpath("/html/body/div[*]/div[2]/div[5]/div[2]")
#     next_button.click()

#     reviews_bj = pd.DataFrame(temp, columns= ["full"])

#     sleep(3)

# reviews = reviews_bj.copy()

In [123]:
# reviews["split"] =reviews["full"].str.split("\n")
# reviews["length"] = [len(i) for i in reviews.split]

In [124]:
# # Not all reviews are the same. Some missing elements
# reviews.length.value_counts()
# reviews.loc[reviews.length == 8,"split"]#[4141]
# reviews.loc[reviews.length == 7,"split"]#[600]
# reviews.loc[reviews.length == 6,"split"]#[0]

8    5357
7      16
6       2
Name: length, dtype: int64

In [126]:
# reviews["title"] = ""
# reviews["subtitle"] = ""
# reviews["date_published"] = ""
# reviews["text"] = ""
# reviews["date_scraped"] = ""
# reviews["rating"] = ""

In [128]:
# # parsing out split to approriate column based on length

# for i, split in enumerate(reviews.split):

#     if reviews.length[i] == 8:
#         reviews.loc[reviews.index == i,"title"] = split[1]
#         reviews.loc[reviews.index == i,"subtitle"] = split[2]
#         reviews.loc[reviews.index == i,"date_published"] = split[3]
#         reviews.loc[reviews.index == i,"text"] = split[4]
#         reviews.loc[reviews.index == i,"date_scraped"] =split[6]
#         reviews.loc[reviews.index == i,"rating"] =split[7]

#     elif len(split) == 7:
#         reviews.loc[reviews.index == i,"title"] = split[1]
#         reviews.loc[reviews.index == i,"subtitle"] = split[2]
#         reviews.loc[reviews.index == i,"date_published"] = split[3]
#         reviews.loc[reviews.index == i,"text"] = "None"
#         reviews.loc[reviews.index == i,"date_scraped"] =split[5]
#         reviews.loc[reviews.index == i,"rating"] =split[6]

#     elif len(split) == 6:
#         reviews.loc[reviews.index == i,"title"] = "None"
#         reviews.loc[reviews.index == i,"subtitle"] = split[0]
#         reviews.loc[reviews.index == i,"date_published"] = split[1]
#         reviews.loc[reviews.index == i,"text"] = split[2]
#         reviews.loc[reviews.index == i,"date_scraped"] =split[4]
#         reviews.loc[reviews.index == i,"rating"] =split[5]

In [130]:
# Checked reviews 
# reviews.loc[reviews.length == 8,]
reviews.loc[reviews.length == 7,]
# reviews.loc[reviews.length == 6,]

,full,split,length,title,subtitle,date_published,text,date_scraped,rating
4170,"2 ratings2 reviews\nNovember 18, 2014\nWorth i...","[2 ratings2 reviews, November 18, 2014, Worth ...",6,None,2 ratings2 reviews,"November 18, 2014",Worth it,04/07/2021 13:35:28,5
4706,"6 ratings3 reviews\nMarch 14, 2014\nBj's has r...","[6 ratings3 reviews, March 14, 2014, Bj's has ...",6,None,6 ratings3 reviews,"March 14, 2014",Bj's has really improved. Great selection of g...,04/07/2021 13:38:46,5


In [139]:
# # Found an anomly with reviews with length 6. Some have no rating but have text. Some have no text but ratings. Some have no text or ratings but are a TOP/HELPFUL Reviewer.

# no_subtitle = reviews.index[reviews.subtitle.str.contains("rating|reviews") == False].tolist()
# no_title = reviews.index[reviews.title.str.contains("rating")].tolist()

# for i, split in enumerate(reviews.split):

#     if len(split) == 7 and (i in no_subtitle):
#         reviews.loc[reviews.index == i,"title"] = split[1]
#         reviews.loc[reviews.index == i,"subtitle"] = "None"
#         reviews.loc[reviews.index == i,"date_published"] = split[2]
#         reviews.loc[reviews.index == i,"text"] = split[3]
#         reviews.loc[reviews.index == i,"date_scraped"] =split[5]

#     if len(split) == 7 and (i in no_title):
#         reviews.loc[reviews.index == i,"title"] = "None" + split[0]
#         reviews.loc[reviews.index == i,"subtitle"] = split[1]
#         reviews.loc[reviews.index == i,"date_published"] = split[2]
#         reviews.loc[reviews.index == i,"text"] = split[3]
#         reviews.loc[reviews.index == i,"date_scraped"] = split[5]

# # Check 
# reviews.loc[reviews.length == 7,]


,full,split,length,title,subtitle,date_published,text,date_scraped,rating
627,T\nTenielleTOP REVIEWER\n15 ratings7 reviews\n...,"[T, TenielleTOP REVIEWER, 15 ratings7 reviews,...",7,TenielleTOP REVIEWER,15 ratings7 reviews,"July 25, 2019",None,04/07/2021 13:13:15,5
629,"L\nLeslie\n2 ratings1 reviews\nJuly 25, 2019\n...","[L, Leslie, 2 ratings1 reviews, July 25, 2019,...",7,Leslie,2 ratings1 reviews,"July 25, 2019",None,04/07/2021 13:13:15,5
636,"M\nMARIA\n12 ratings4 reviews\nJuly 25, 2019\n...","[M, MARIA, 12 ratings4 reviews, July 25, 2019,...",7,MARIA,12 ratings4 reviews,"July 25, 2019",None,04/07/2021 13:13:19,5
690,"M\nMadeleyne\n1 ratings1 reviews\nJuly 15, 201...","[M, Madeleyne, 1 ratings1 reviews, July 15, 20...",7,Madeleyne,1 ratings1 reviews,"July 15, 2019",None,04/07/2021 13:13:42,5
733,"S\nSucheta\n5 ratings3 reviews\nJuly 4, 2019\n...","[S, Sucheta, 5 ratings3 reviews, July 4, 2019,...",7,Sucheta,5 ratings3 reviews,"July 4, 2019",None,04/07/2021 13:13:57,4
744,"A\nAnna\n1 ratings1 reviews\nJuly 2, 2019\nHel...","[A, Anna, 1 ratings1 reviews, July 2, 2019, He...",7,Anna,1 ratings1 reviews,"July 2, 2019",None,04/07/2021 13:14:01,5
760,M\nMichaelHELPFUL REVIEWER\n11 ratings4 review...,"[M, MichaelHELPFUL REVIEWER, 11 ratings4 revie...",7,MichaelHELPFUL REVIEWER,11 ratings4 reviews,"June 24, 2019",None,04/07/2021 13:14:08,5
800,"A\nAlberto\n1 ratings1 reviews\nJune 9, 2019\n...","[A, Alberto, 1 ratings1 reviews, June 9, 2019,...",7,Alberto,1 ratings1 reviews,"June 9, 2019",None,04/07/2021 13:14:23,5
827,"S\nSascha\n10 ratings4 reviews\nJune 1, 2019\n...","[S, Sascha, 10 ratings4 reviews, June 1, 2019,...",7,Sascha,10 ratings4 reviews,"June 1, 2019",None,04/07/2021 13:14:30,5
895,"N\nNick\n1 ratings1 reviews\nMay 10, 2019\nHel...","[N, Nick, 1 ratings1 reviews, May 10, 2019, He...",7,Nick,1 ratings1 reviews,"May 10, 2019",None,04/07/2021 13:14:56,5


In [140]:
# # # Parsing out title and subtitle columns further

# reviews["top_reviewer"] = [1 if ("TOP REVIEWER" in i) else 0 for i in reviews.title]
# reviews["helpful_reviewer"] = [1 if ("HELPFUL REVIEWER" in i) else 0 for i in reviews.title]
# reviews["rating_count"] = [int(re.findall(r'\d+',i)[0]) if len(re.findall(r'\d+',i))>0 else 0 for i in reviews.subtitle]
# reviews["name"] = [re.split('TOP REVIEWER|HELPFUL REVIEWER',i)[0] for i in reviews.title]

# reviews["review_count"] = 0
# for i, subtitle in enumerate(reviews.subtitle[:10]):
#     num = re.findall(r'\d+',subtitle)
#     if len(num) > 0:
#         if search ("rating", subtitle):
#             reviews.loc[reviews.index == i,"review_count"] = int(num[1])
#         else:
#             reviews.loc[reviews.index == i,"review_count"] = int(num[0])

In [141]:
# # Formating date_published that are in a relative format (# days/hours ago)

# for i, date in enumerate(reviews.date_published):
    
#     if "ago" in date:
#         if "day" in date:
#             scraped = datetime.strptime(reviews.date_scraped[i],"%m/%d/%Y %H:%M:%S").date()
#             ago = timedelta(days = int(re.findall(r'\d+',date)[0]))
#             reviews.loc[reviews.index == i,"date_published"] = (scraped - ago).strftime("%m/%d/%Y")
#         else:
#             reviews.loc[reviews.index == i,"date_published"] = reviews.date_scraped[i][:10]
#     elif date != "'": 
#         reviews.loc[reviews.index == i,"date_published"] = datetime.strptime(reviews.date_published[i],"%B %d, %Y").date().strftime("%m/%d/%Y")


In [142]:
reviews

,full,split,length,title,subtitle,date_published,text,date_scraped,rating,top_reviewer,helpful_reviewer,rating_count,name,review_count
0,"E\nEllen\n2 ratings2 reviews\nMarch 20, 2021\n...","[E, Ellen, 2 ratings2 reviews, March 20, 2021,...",8,Ellen,2 ratings2 reviews,03/20/2021,I’ve always appreciated BJ’s and shopped at a ...,04/07/2021 13:09:03,3,0,0,2,Ellen,2
1,H\nHeatherTOP REVIEWERHELPFUL REVIEWER\n33 rat...,"[H, HeatherTOP REVIEWERHELPFUL REVIEWER, 33 ra...",8,HeatherTOP REVIEWERHELPFUL REVIEWER,33 ratings28 reviews,03/17/2021,Glad I got this. I went there today and got so...,04/07/2021 13:09:03,5,1,1,33,Heather,28
2,J\nJulietteTOP REVIEWER\n13 ratings9 reviews4 ...,"[J, JulietteTOP REVIEWER, 13 ratings9 reviews4...",8,JulietteTOP REVIEWER,13 ratings9 reviews4 photos,02/24/2021,Couldn’t print out a physical card and gave me...,04/07/2021 13:09:03,1,1,0,13,Juliette,9
3,"E\nErika\n2 ratings1 reviews\nFebruary 4, 2021...","[E, Erika, 2 ratings1 reviews, February 4, 202...",8,Erika,2 ratings1 reviews,02/04/2021,"It was very easy, you just show them the barco...",04/07/2021 13:09:03,5,0,0,2,Erika,1
4,"L\nLinda\n1 ratings1 reviews\nFebruary 3, 2021...","[L, Linda, 1 ratings1 reviews, February 3, 202...",8,Linda,1 ratings1 reviews,02/03/2021,"Is a great place for shopping for everyone, es...",04/07/2021 13:09:03,5,0,0,1,Linda,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5370,A\nAngelaTOP REVIEWER\n7 ratings6 reviews\nAug...,"[A, AngelaTOP REVIEWER, 7 ratings6 reviews, Au...",8,AngelaTOP REVIEWER,7 ratings6 reviews,08/24/2013,Can double coupons here compared to other,04/07/2021 13:43:11,5,1,0,7,Angela,0
5371,T\nTrennyTOP REVIEWER\n65 ratings18 reviews\nA...,"[T, TrennyTOP REVIEWER, 65 ratings18 reviews, ...",8,TrennyTOP REVIEWER,65 ratings18 reviews,08/23/2013,Best groupon yet!,04/07/2021 13:43:11,5,1,0,65,Trenny,0
5372,"J\nJeremy\n3 ratings1 reviews\nAugust 23, 2013...","[J, Jeremy, 3 ratings1 reviews, August 23, 201...",8,Jeremy,3 ratings1 reviews,08/23/2013,This was a great deal and everyone needs to ta...,04/07/2021 13:43:11,5,0,0,3,Jeremy,0
5373,"G\nGladys\n5 ratings3 reviews\nAugust 22, 2013...","[G, Gladys, 5 ratings3 reviews, August 22, 201...",8,Gladys,5 ratings3 reviews,08/22/2013,Good deal.,04/07/2021 13:43:11,5,0,0,5,Gladys,0


In [143]:
# # BJ's CSV
# reviews.to_csv('Groupon_BJs_Raw.csv', index=False)
# reviews[['name', 'text', 'rating', 'review_count','top_reviewer', 'helpful_reviewer','date_published','date_scraped']].to_csv('Groupon_BJs.csv', index=False)

# Sam's Club Data Collection

In [84]:
# PATH = "C:\Program Files (x86)\chromedriver.exe"
# driver = webdriver.Chrome(PATH)
# driver.get("https://www.groupon.com/deals/n-sams-club-membership-packages-7#tips") #Sam's Club
# driver.maximize_window()

# sleep(4)

# all_reviews = driver.find_element_by_xpath("/html/body/section[2]/div/section/div/div[2]/div[2]/section/div/div/article/div[3]/div[2]/div[2]/a")
# all_reviews.click()

In [94]:
#temp = []
# for i in range(1732): #2859
    
#     for j in scrape():
#         temp.append(j)
#     next_button = driver.find_element_by_xpath("/html/body/div[*]/div[2]/div[5]/div[2]")
#     next_button.click()

#     reviews_sam = pd.DataFrame(temp, columns= ["full"])

#     sleep(3)

# reviews = reviews_sam.copy()

In [96]:
# reviews["split"] =reviews["full"].str.split("\n")
# reviews["length"] = [len(i) for i in reviews.split]

In [97]:
# # Not all reviews are the same. Some missing elements
# reviews.length.value_counts()
# reviews.loc[reviews.length == 8,"split"]
# reviews.loc[reviews.length == 7,"split"]
# reviews.loc[reviews.length == 6,"split"]

8    28400
7      122
6       18
Name: length, dtype: int64

In [99]:
# reviews["title"] = ""
# reviews["subtitle"] = ""
# reviews["date_published"] = ""
# reviews["text"] = ""
# reviews["date_scraped"] = ""
# reviews["rating"] = ""

In [118]:
# for i, split in enumerate(reviews.split):

#     if len(split) == 8:
#         reviews.loc[reviews.index == i,"title"] = split[1]
#         reviews.loc[reviews.index == i,"subtitle"] = split[2]
#         reviews.loc[reviews.index == i,"date_published"] = split[3]
#         reviews.loc[reviews.index == i,"text"] = split[4]
#         reviews.loc[reviews.index == i,"date_scraped"] =split[6]
#         reviews.loc[reviews.index == i,"rating"] =split[7]

#     elif len(split) == 7:
#         reviews.loc[reviews.index == i,"title"] = split[1]
#         reviews.loc[reviews.index == i,"subtitle"] = split[2]
#         reviews.loc[reviews.index == i,"date_published"] = split[3]
#         reviews.loc[reviews.index == i,"text"] = "None"
#         reviews.loc[reviews.index == i,"date_scraped"] =split[5]
#         reviews.loc[reviews.index == i,"rating"] =split[6]

#     elif len(split) == 6:
#         reviews.loc[reviews.index == i,"title"] = "None"
#         reviews.loc[reviews.index == i,"subtitle"] = split[0]
#         reviews.loc[reviews.index == i,"date_published"] = split[1]
#         reviews.loc[reviews.index == i,"text"] = split[2]
#         reviews.loc[reviews.index == i,"date_scraped"] =split[4]
#         reviews.loc[reviews.index == i,"rating"] =split[5]


In [117]:
# Checked reviews 
# reviews.loc[reviews.length == 8,]
# reviews.loc[reviews.length == 7,]
# reviews.loc[reviews.length == 6,]

In [116]:
# # Found an anomly with reviews with length 6. Some have no rating but have text. Some have no text but ratings. Some have no text or ratings but are a TOP/HELPFUL Reviewer.

# no_subtitle = reviews.index[reviews.subtitle.str.contains("rating|reviews") == False].tolist()
# no_title = reviews.index[reviews.title.str.contains("rating")].tolist()

# for i, split in enumerate(reviews.split):

#     if len(split) == 7 and (i in no_subtitle):
#         reviews.loc[reviews.index == i,"title"] = split[1]
#         reviews.loc[reviews.index == i,"subtitle"] = "None"
#         reviews.loc[reviews.index == i,"date_published"] = split[2]
#         reviews.loc[reviews.index == i,"text"] = split[3]
#         reviews.loc[reviews.index == i,"date_scraped"] =split[5]

#     if len(split) == 7 and (i in no_title):
#         reviews.loc[reviews.index == i,"title"] = "None" + split[0]
#         reviews.loc[reviews.index == i,"subtitle"] = split[1]
#         reviews.loc[reviews.index == i,"date_published"] = split[2]
#         reviews.loc[reviews.index == i,"text"] = split[3]
#         reviews.loc[reviews.index == i,"date_scraped"] = split[5]

# # Check 
# reviews.loc[reviews.length == 7,]

In [110]:
# # # Parsing out title and subtitle columns further

# reviews["top_reviewer"] = [1 if ("TOP REVIEWER" in i) else 0 for i in reviews.title]
# reviews["helpful_reviewer"] = [1 if ("HELPFUL REVIEWER" in i) else 0 for i in reviews.title]
# reviews["rating_count"] = [int(re.findall(r'\d+',i)[0]) if len(re.findall(r'\d+',i))>0 else 0 for i in reviews.subtitle]
# reviews["name"] = [re.split('TOP REVIEWER|HELPFUL REVIEWER',i)[0] for i in reviews.title]

# reviews["review_count"] = 0
# for i, subtitle in enumerate(reviews.subtitle[:10]):
#     num = re.findall(r'\d+',subtitle)
#     if len(num) > 0:
#         if search ("rating", subtitle):
#             reviews.loc[reviews.index == i,"review_count"] = int(num[1])
#         else:
#             reviews.loc[reviews.index == i,"review_count"] = int(num[0])

In [111]:
# # Formating date_published that are in a relative format (# days/hours ago)

# for i, date in enumerate(reviews.date_published):
    
#     if "ago" in date:
#         if "day" in date:
#             scraped = datetime.strptime(reviews.date_scraped[i],"%m/%d/%Y %H:%M:%S").date()
#             ago = timedelta(days = int(re.findall(r'\d+',date)[0]))
#             reviews.loc[reviews.index == i,"date_published"] = (scraped - ago).strftime("%m/%d/%Y")
#         else:
#             reviews.loc[reviews.index == i,"date_published"] = reviews.date_scraped[i][:10]
#     else: 
#         reviews.loc[reviews.index == i,"date_published"] = datetime.strptime(reviews.date_published[i],"%B %d, %Y").date().strftime("%m/%d/%Y")


In [115]:
# # # Sam's Club CSV
# reviews.to_csv('Groupon_SamsClub_Raw.csv', index=False)
# reviews[['name', 'text', 'rating', 'review_count','top_reviewer', 'helpful_reviewer','date_published','date_scraped']].to_csv('Groupon_SamsClub.csv', index=False)